In [ ]:
import os
import emcee
import corner

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy import stats
from brick.azr import AZR
from multiprocess import Pool

# Ignore RuntimeWarning
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Restrict processes to one thread only
os.environ['OMP_NUM_THREADS'] = '1'

# Define the parameters prior distributions
priors = [
     # ANC 1/2+ 0 keV
    stats.norm(1.63,0.12),

    # 2.35 MeV Resonance
    stats.uniform(2.30, 0.10), stats.uniform(0, 1e6), stats.uniform(-10, 20),
    
    # Its background pole
    stats.uniform(-1e8,2e8),

    # 3.50 MeV Resonance
    stats.uniform(3.45, 0.10), stats.uniform(0, 1e6), stats.uniform(-10, 20), stats.uniform(-10, 20),
    
    # Its background pole
    stats.uniform(-1e8,2e8), stats.uniform(-1e8,2e8),

    # 3.55 MeV Resonance
    stats.uniform(3.50, 0.10), stats.uniform(0, 1e6),
    
    # Normalization Parameters
    stats.lognorm(0.05),     # Meyer
    stats.lognorm(0.069),    # Skowronski PRL - HPGe
    stats.lognorm(0.079),    # Skowronski PRL - BGO
    stats.lognorm(0.085),    # Skowronski PRC
    stats.lognorm(0.06),     # Gyurky
    stats.lognorm(0.10),     # Kettner
    stats.lognorm(0.10),     # Burtebaev
    stats.uniform(0,2),      # Vogl 
    stats.uniform(0,2),      # Rolfs
    stats.uniform(0,2),      # Bailey
    stats.uniform(0,2)       # Lamb
]

# Minimization variables
nsteps   = 1000         # How many steps should each walker take?
nprocs   = 5            # How many Python processes do you want to allocate?
ndim     = len(priors)  # How many parameters are you fitting?
nwalkers = 50           # How many walkers do you want to use?

In [ ]:
# We read the .azr file
azr = AZR('12c_pg.azr')
azr.ext_capture_file='output/intEC.dat'

In [ ]:
# We get the initial values from AZURE2
theta0 = azr.config.get_input_values()
ntheta = len(theta0)

# We now need to do a trick to get the data files
output = np.vstack( azr.predict( theta0, dress_up=False ) )

# Now we loop over the data files to take their order and length and cut the output
index, data = 0, { }
for segment in azr.config.data.segments:
    data[segment.filename] = output[index:index+len(segment.values)]
    index += len(segment.values)

# We get the data from the prediction
output  = np.vstack( azr.predict( theta0, dress_up=False ) )
x       = output[:, 0]
y       = output[:, 5]
dy_bare = output[:, 6]

In [ ]:
# Prior log probability
def lnPi( theta ):
    return np.sum([pi.logpdf(t) for (pi, t) in zip(priors, theta)])  # Get the prior probability

# Log likelihood
def lnL( theta ):
    output = np.vstack(azr.predict(theta, dress_up=False))                      # Get prediction from AZURE2
    mu, y, dy = output[:, 3], output[:, 5], output[:, 6]                        # Unpack the data
    lnl = np.sum(-0.5*np.log(2*np.pi*pow(dy_bare,2)) - 0.5*pow((y - mu)/dy,2))  # Calculate the likelihood
    return lnl

# Posterior log probability
def lnP( theta ):
    lnpi = lnPi( theta )                        # Calculate the prior
    if not np.isfinite( lnpi ): return -np.inf  # If the prior is not finite, return -inf
    lnl = lnL( theta )                          # Calculate the likelihood
    return lnl + lnpi

In [ ]:
# Prepare initial walker positions (50% region around the initial values)
p0 = np.zeros( (nwalkers, ndim) )
for i in range(nwalkers):
    for j in range(ndim):
        if( j not in [1, 5, 11] ): p0[i,j] = np.sign(theta0[j]) * stats.uniform( abs(theta0[j]) * 0.95, abs(theta0[j]) * 0.1 ).rvs()
        else: p0[i,j] = np.sign(theta0[j]) * stats.uniform( abs(theta0[j]) * 0.99, abs(theta0[j]) * 0.02 ).rvs()

# Prepare initial walkers position from the "results/best-leastsq.txt" file
with open('results/best-leastsq.txt') as f:
    lines = (line for line in f if not line.startswith('#'))
    p0 = np.loadtxt(lines, usecols=(1))
cov = np.loadtxt('results/covariance-leastsq.txt')
p0 = np.random.multivariate_normal(p0, cov, nwalkers)

In [ ]:
# Prepare the file to write the chains
backend = emcee.backends.HDFBackend('chains/samples.h5') 
backend.reset(nwalkers, len(priors))

# For a stretch move
move = emcee.moves.StretchMove( )

# For a walk move
#move = emcee.moves.WalkMove( )

# For a gaussian Metropolis-Hastings move
#move = emcee.moves.GaussianMove( 0.1 * cov )

# Run the sampling
with Pool(processes=nprocs) as pool:
    sampler = emcee.EnsembleSampler( nwalkers, ndim, lnP, pool=pool, backend=backend, moves=[move] ) 
    state = sampler.run_mcmc( p0, nsteps, progress=True, tune=True )

In [ ]:
backend = emcee.backends.HDFBackend('chains/samples.h5')

# Get the number of samples after burn in
nb  = 300
lnp = backend.get_log_prob(discard=nb)
ii  = np.where(np.median(lnp, axis=0)>-40000)[0]

# Get only the first chain
samples = backend.get_chain(discard=nb)[:, ii, :]
samples = samples.reshape((-1, samples.shape[-1]))

In [ ]:
# Plot the probability
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.plot(lnp)

ax.set_xlabel('Step')
ax.set_ylabel('ln( P )')

plt.show()

In [ ]:
# Plot the parameters
fig, ax = plt.subplots( int(ntheta / 2) + 1, 2, figsize=( 20, 70 ) )

means, lows, highs = [], [], []
for i in range( ntheta ):

        mean = np.percentile( samples[:,i], 50 )
        low = np.percentile( samples[:,i], 16 )
        high = np.percentile( samples[:,i], 84 )
    
        ax[i//2, i%2].set_title( azr.config.labels[i] )
        ax[i//2, i%2].set_xlabel( "Step" )

        ax[i//2, i%2].plot( samples[:,i], color="tab:blue" )
        ax[i//2, i%2].axhline( mean, color="tab:red", linestyle="-", lw=4 )
        ax[i//2, i%2].axhline( low, color="tab:red", linestyle="--", lw=2 )
        ax[i//2, i%2].axhline( high, color="tab:red", linestyle="--", lw=2 )

        ax[i//2, i%2].set_ylabel( "Value" )

        means.append( mean )
        lows.append( low )
        highs.append( high )

In [ ]:
# Save to file
with open( "results/best-emcee.txt", "w" ) as f:
    for i in range( ntheta ):
        f.write( f"{azr.config.labels[i]}: {means[i]:.5f} + {highs[i]-means[i]:.5f} - {means[i]-lows[i]:.5f}\n" )

In [ ]:
# Plot a corner plot
fig = plt.figure(figsize=(8, 6))

corner.corner(samples, labels=[f'p{i}' for i in range(len(priors))])

plt.show( )

In [ ]:
# Plot the correlation matrix
fig, ax = plt.subplots(1, 1, figsize=(11, 9))

correlation = np.corrcoef(samples.T)

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(correlation, dtype=bool))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlation, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

# Set title and labels
plt.title("Correlation matrix")

# Change xticks and yticks
plt.xticks(range(len(theta0)), azr.config.labels, rotation=90)
plt.yticks(range(len(theta0)), azr.config.labels, rotation=0)

plt.show( )

In [ ]:
# Calculate the covariance
covariance = np.cov(samples.T)

# Write the covariance to a file
with open( "results/covariance-emcee.txt", "w" ) as f:
    for i in range(len(theta0)):
        for j in range(len(theta0)):
            f.write("{:15.6f} ".format(covariance[i,j]))
        f.write("\n")

In [ ]:
# Function to cut the extrapolations between pp and pg channels
def cut( result ):
    samples = [ ]
    matrix = np.array( result )
    # Cut array in the points where the first column, thus the energy, decrease
    indexes = np.where( np.diff( matrix[:,0] ) < 0 )[0]
    for i in range( len(indexes) + 1 ):
        if( i == 0 ): samples.append( matrix[:indexes[i],] )
        elif( i == len(indexes) ): samples.append( matrix[indexes[i-1]+1:,] )
        else: samples.append( matrix[indexes[i-1]+1:indexes[i],] )
    return samples

# Calculate the xtrapolations
buckets = { "integral"           : [ ],
            "differential_0deg"  : [ ],
            "differential_55deg" : [ ],
            "differential_90deg" : [ ],
            "elastic_84deg"      : [ ],
            "elastic_114deg"     : [ ],
            "elastic_144deg"     : [ ] }

# Shuffle the samples before extrapolating
np.random.shuffle( samples )

# We will sample only last 1000 samples for time reasons
for sample in tqdm( samples[-1000:] ):

    # Extrapolate gives an array of extrapolations for each channel
    result = azr.extrapolate( sample, ext_capture_file="output/intEC.extrap" )

    # In each channel, the data are not divided per segment
    samples_pp = cut( result[0] )
    samples_pg = cut( result[1] )

    # Append the extrapolation to each segment in AZURE2 order
    buckets["integral"].append( samples_pg[0] )
    buckets["differential_0deg"].append( samples_pg[1] )
    buckets["differential_55deg"].append( samples_pg[2] )
    buckets["differential_90deg"].append( samples_pg[3] )
    buckets["elastic_84deg"].append( samples_pp[0] )
    buckets["elastic_114deg"].append( samples_pp[1] )
    buckets["elastic_144deg"].append( samples_pp[2] )

In [ ]:
# Calculate the correlation between each parameter and each cross section at each energy
correlations = {}
for key in buckets.keys( ):
    array = np.array( buckets[key] )
    #print( buckets[key][0][0], array[0][0] )
    correlations[key] = np.zeros( (len(theta0), len( array[0][:,0] )) )
    for i in range( len(theta0) ):
        for k in range( len( array[0][:,0] ) ):
            corr = np.corrcoef( samples[-1000:, i], array[:,k][:,4] )[0, 1]
            correlations[key][i][k] = corr

In [ ]:
# Plot the correlation coefficient for each energy point
fig, ax = plt.subplots( len( theta0 ), len( buckets ), figsize=(60, 170) )
for i, key in enumerate( buckets.keys( ) ):
    for j in range( len( theta0 ) ):
        x, y = buckets[key][0][:,0], correlations[key][j]
        cmap = sns.color_palette("coolwarm", as_cmap=True)
        ax[j][i].scatter( x, y, c=y, cmap=cmap, vmin=-1, vmax=1, s=20 )
        ax[j][i].set_title( key )
        ax[j][i].set_xlabel( "Energy (MeV)" )
        ax[j][i].set_ylabel( r"Correlation of {}".format( azr.config.labels[j] ) )
        ax[j][i].set_xlim( 0, x[-1] )

In [ ]:
# Plotting the cross sections
fig, ax = plt.subplots( 3, 3, figsize=( 20, 20 ) )

for i, key in enumerate( buckets.keys( ) ):

    bucket = np.array( buckets[key] )

    # Calculate the mean and standard deviation
    mean = np.mean( bucket, axis=0 )
    std  = np.std( bucket, axis=0 )
    
    ax[i//3, i%3].set_title( key )
    ax[i//3, i%3].set_xlabel( "Energy (MeV)" )
    ax[i//3, i%3].set_ylabel( "S-factor (MeV b)" )

    ax[i//3, i%3].plot( mean[:,0], mean[:,4], label="Extrapolation", color="red" )
    ax[i//3, i%3].fill_between( mean[:,0], mean[:,4] - std[:,4], mean[:,4] + std[:,4], color="red", alpha=0.5 )

    if( key == "integral" ): 
        ax[i//3, i%3].errorbar( data["skowronski_luna_hpge.dat"][:,0], data["skowronski_luna_hpge.dat"][:,4], yerr=data["skowronski_luna_hpge.dat"][:,5], label="LUNA HPGe", fmt="o" )
        ax[i//3, i%3].errorbar( data["skowronski_luna_bgo.dat"][:,0], data["skowronski_luna_bgo.dat"][:,4], yerr=data["skowronski_luna_bgo.dat"][:,5], label="LUNA BGO", fmt="o" )
        ax[i//3, i%3].errorbar( data["skowronski_fels.dat"][:,0], data["skowronski_fels.dat"][:,4], yerr=data["skowronski_fels.dat"][:,5], label="LUNA BGO", fmt="o" )
        ax[i//3, i%3].errorbar( data["vogl.dat"][:,0], data["vogl.dat"][:,7], yerr=data["vogl.dat"][:,8], label="Vogl", fmt="o" )
        ax[i//3, i%3].errorbar( data["gyurky.dat"][:,0], data["gyurky.dat"][:,7], yerr=data["gyurky.dat"][:,8], label="Gyurky", fmt="o" )
        ax[i//3, i%3].errorbar( data["burtebaev.dat"][:,0], data["burtebaev.dat"][:,7], yerr=data["burtebaev.dat"][:,8], label="Burtebaev", fmt="o" )
        ax[i//3, i%3].errorbar( data["lamb.dat"][:,0], data["lamb.dat"][:,7], yerr=data["lamb.dat"][:,8], label="Vogl", fmt="o" )
        ax[i//3, i%3].errorbar( data["bailey.dat"][:,0], data["bailey.dat"][:,7], yerr=data["bailey.dat"][:,8], label="Vogl", fmt="o" )
        ax[i//3, i%3].set_xlim(0, 2.5)

    if( key == "differential_0deg" ):
        mask = (data["rolfs.dat"][:,2] == 0)
        ax[i//3, i%3].errorbar( data["rolfs.dat"][mask][:,0], data["rolfs.dat"][mask][:,7], yerr=data["rolfs.dat"][mask][:,8], label="Rolfs", fmt="o" )
        mask = (data["kettner.dat"][:,2] == 0)
        ax[i//3, i%3].errorbar( data["kettner.dat"][mask][:,0], data["kettner.dat"][mask][:,7], yerr=data["kettner.dat"][mask][:,8], label="Kettner", fmt="o" )
        ax[i//3, i%3].set_xlim(0, 2.5)

    if( key == "differential_55deg" ):
        mask = (data["kettner.dat"][:,2] == 55)
        ax[i//3, i%3].errorbar( data["kettner.dat"][mask][:,0], data["kettner.dat"][mask][:,7], yerr=data["kettner.dat"][mask][:,8], label="Kettner", fmt="o" )
        ax[i//3, i%3].set_xlim(0, 2.5)

    if( key == "differential_90deg" ):
        mask = (data["rolfs.dat"][:,2] == 90)
        ax[i//3, i%3].errorbar( data["rolfs.dat"][mask][:,0], data["rolfs.dat"][mask][:,7], yerr=data["rolfs.dat"][mask][:,8], label="Rolfs", fmt="o" )
        ax[i//3, i%3].set_xlim(0, 2.5)

    if( key == "elastic_84deg" ):
        mask = (data["meyer.dat"][:,2] == 89.09121)
        ax[i//3, i%3].errorbar( data["meyer.dat"][mask][:,0], data["meyer.dat"][mask][:,7], yerr=data["meyer.dat"][mask][:,8], label="Meyer", fmt="o" )
        ax[i//3, i%3].set_ylim(1e0, 1e5)

    if( key == "elastic_114deg" ):
        mask = (data["meyer.dat"][:,2] == 118.8806)
        ax[i//3, i%3].errorbar( data["meyer.dat"][mask][:,0], data["meyer.dat"][mask][:,7], yerr=data["meyer.dat"][mask][:,8], label="Meyer", fmt="o" )

    if( key == "elastic_144deg" ):
        mask = (data["meyer.dat"][:,2] == 146.9212)
        ax[i//3, i%3].errorbar( data["meyer.dat"][mask][:,0], data["meyer.dat"][mask][:,7], yerr=data["meyer.dat"][mask][:,8], label="Meyer", fmt="o" )
        ax[i//3, i%3].set_ylim(1e-1, 1e5)

    ax[i//3, i%3].legend( )
    ax[i//3, i%3].set_yscale( "log" )